# Video Recommend Model
## 1. 모델 훈련
konlpy의 morpheme tokenizer와 tfidf vectorizer를 사용하여 콘텐츠 기반 필터링 모델을 구현합니다.
- __init__ : vectorizer & tokenizer 객체 초기화
- tokenize : 주어진 문장을 형태소 단위의 토큰으로 나누어 리스트에 저장하는 함수
- vectorize : 토큰화된 문장을 리스트에 저장하는 함수 (vectorization의 input으로 들어갈 리스트 생성)
- fit_transform : 주어진 csv 데이터를 통해 tfidf matrix와 cosine_sim matrix를 생성하는 함수
- transform : 새로운 비디오에 대한 cosine_sim matrix를 반환하는 함수
- predict : 특정 비디오들과 유사한 비디오 리스트를 반환하는 함수

In [214]:
#==================== Import Packages ====================#
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from konlpy.tag import Okt

In [215]:
class content_based_filtering:

    def __init__(self):
        self.tfidf_vectorizer = TfidfVectorizer(stop_words="english", lowercase=True)
        self.okt = Okt()
        self.tfidf_matrix = None
        self.cosine_sim = None
        self.videos_df = None

    def tokenize(self, contents):
        contents_tokens = [self.okt.morphs(row) for row in contents]
        return contents_tokens

    def vectorize(self, contents_tokens):
        contents_vecs = []
        for tokens in contents_tokens:
            sentence = ""
            for token in tokens:
                sentence += ' ' + token
            contents_vecs.append(sentence)
        return contents_vecs

    def fit_transform(self, filename):
        videos_df = pd.read_csv(filename)
        videos_df["title"] = videos_df["title"].fillna("")
        videos_df["description"] = videos_df["description"].fillna("")
        videos_df = videos_df[videos_df['title'] != 'Private video']
        videos_df['text'] = videos_df["title"] + " " + videos_df["description"]
        self.videos_df = videos_df[["text", "videoId", "title"]]

        contents_tokens = self.tokenize(self.videos_df["text"])
        contents_vecs = self.vectorize(contents_tokens)
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(contents_vecs)
        self.cosine_sim = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)

    def transform(self, input_videos):
        contents_tokens = self.tokenize(self.videos_df["text"])
        contents_vecs = self.vectorize(contents_tokens)
        input_tfidf_matrix = self.tfidf_vectorizer.transform(contents_vecs)
        cosine_sim = linear_kernel(input_tfidf_matrix, input_tfidf_matrix)
        return cosine_sim

    def predict(self, video_ids):

        final_video_indices = []
        for video_id in video_ids:

            try:
                idx = self.videos_df[self.videos_df['videoId'] == video_id].index[0]

            except IndexError:
                print("null")
                return None

            # 해당 영상에 대한 유사도 측정
            sim_scores = list(enumerate(self.cosine_sim[idx]))

            # 유사도에 따라 정렬
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # 상위 10개 영상 선택
            sim_scores = sim_scores[1:11]

            # 선택된 영상의 인덱스
            final_video_indices += [i[0] for i in sim_scores]

        # 선택된 영상의 제목으로 반환
        final_video_indices = list(set(final_video_indices))[:10]
        return self.videos_df['title'].iloc[final_video_indices].tolist()

In [216]:
cbf=content_based_filtering()
cbf.fit_transform("./kbo_video.csv")
result = cbf.predict(["AfPaDw2upuk", "Dru7TvHX7lY"])
print(result)
print(cbf.cosine_sim)

['서건창의 날 (240403)', '네일 10승! 기아 VS 키움 8월 13일 하이라이트', '구단 역사상 최다 매진! 스타우트 첫 승!｜ 9월 7일 KIA vs 키움 하이라이트', '2024 한국시리즈 우승콜', '황동하 데뷔 첫 승 경기! 3연승!❤️ | 5월 18일 덕관 | 기아 vs NC', '역전은 백투백이 제맛', '올해 첫 만루홈런의 주인공', '하루 2경기 전승! 🔥｜한국시리즈 2차전 하이라이트', '5연승 질주! 8월 21일 KIA vs 롯데 경기 하이라이트', '14일 SSG전 역전 순간!']
[[1.         0.05985762 0.11879739 ... 0.         0.01767005 0.        ]
 [0.05985762 1.         0.1580122  ... 0.         0.         0.        ]
 [0.11879739 0.1580122  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.04017336 0.08090779]
 [0.01767005 0.         0.         ... 0.04017336 1.         0.1775868 ]
 [0.         0.         0.         ... 0.08090779 0.1775868  1.        ]]


## 2. S3 모델 업로드
훈련된 콘텐츠 기반 필터링 모델을 S3에 pkl 파일로 업로드합니다.
- video_df.pkl : 야구 영상 데이터프레임을 직렬화한 pkl 파일
- cosine_sim.pkl : 야구 영상에 관한 코사인 유사도 행렬을 직렬화한 pkl 파일

In [217]:
import os
import pickle
import tarfile
import boto3
import shutil
import joblib
import logging
import sagemaker

# S3 클라이언트 설정
region = 'ap-northeast-2'
#boto3.setup_default_session(region_name=region)
s3_client = boto3.client('s3')

# SageMaker 역할 및 세션 설정
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()


def upload_model_to_s3(model, bucket_name, model_filename):

    # 임시 디렉토리 생성
    model_dir = "models"  # 상대 경로로 설정
    os.makedirs(model_dir, exist_ok=True)

    # 모델 직렬화 (pkl 파일로 저장)
    #model_pkl_path = os.path.join(model_dir, "content_based_model.pkl")
    model_pkl_path = os.path.join(model_dir, "video_df.pkl")
    with open(model_pkl_path, "wb") as model_file:
        #model_to_save = model.video_df
        model.videos_df.to_pickle(model_file)
    
    model_pkl_path = os.path.join(model_dir, "cosine_sim.pkl")
    with open(model_pkl_path, "wb") as model_file:
        model_to_save = model.cosine_sim.tolist()
        joblib.dump(model_to_save, model_file)

    # predictor.py를 모델 디렉토리에 복사
    predictor_script_path = "predictor.py"
    shutil.copy(predictor_script_path, model_dir)

    # tar.gz 형식으로 압축
    tar_gz_path = model_filename  # 압축할 최종 파일 경로
    with tarfile.open(tar_gz_path, 'w:gz') as tar:
        #tar.add('models/content_based_model.pkl', arcname='content_based_model.pkl')
        tar.add("models/video_df.pkl", arcname="video_df.pkl")
        tar.add("models/cosine_sim.pkl", arcname="cosine_sim.pkl")

    # S3에 압축된 모델 파일 업로드
    s3_path = f"models/{os.path.basename(tar_gz_path)}"
    s3_client.upload_file(tar_gz_path, bucket_name, s3_path)

#bucket_name = sagemaker_session.default_bucket()
bucket_name = "yaong-baseball"
model_name = "models/content_based_model.tar.gz"

upload_model_to_s3(cbf, bucket_name, model_name)

In [219]:
# Check if the file exists in the specified S3 path
try:
    s3_client.head_object(Bucket=bucket_name, Key=model_name)
    print("File exists!")
except s3_client.exceptions.ClientError as e:
    print("File not found:", e)

File exists!
